<a href="https://colab.research.google.com/github/yixish/NLPLearning/blob/master/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')可以

Mounted at /content/gdrive


In [ ]:
!/opt/bin/nvidia-smi

Fri Nov  6 05:04:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# f = open("/content/gdrive/My Drive/dataset/en-cn.csv","r")
# pairs = f.readlines()

import pandas as pd

df = pd.read_csv("/content/gdrive/My Drive/dataset/en-cn.csv",names=['id', 'en','cn'])
ens = df['en'].values
cns = df['cn'].values

pairs = []
num = 0
for i in range(len(ens)):
    if len(str(ens[i]).split()) < 100:
        pairs.append([str(ens[i]),str(cns[i])])
        num+=1
    if num == 128:
        break;

# pairs = pairs[1:100]
# pairs = [ s.replace('\n',"").split('\t') for s in pairs]
pairs[:10]
len(pairs)

# df = pd.DataFrame(np.array(pairs[1:]))
# df.to_csv("/home/kesci/work/sub.csv",header=None)

128

In [ ]:
en_chars = set()
cn_chars = set()


en_sents = []
cn_sents = []

for pair in pairs:
    en_sent = pair[0]
    cn_sent = pair[1]
    en_sents.append(en_sent)
    cn_sents.append(cn_sent)

    for word in en_sent.split():
        en_chars.add(word)

    for word in cn_sent:
        cn_chars.add(word)

en_chars = list(en_chars)
cn_chars = list(cn_chars)

en_chars.extend(['<SOS>',"<EOS>","<PAD>"])
cn_chars.extend(['<SOS>',"<EOS>","<PAD>"])

en_chars = sorted(en_chars)
cn_chars = sorted(cn_chars)

num_encoder_tokens = len(en_chars)
num_decoder_tokens = len(cn_chars)
max_encoder_seq_len = max([ len(s.split()) for s in en_sents])
max_decoder_seq_len = max([ len(s) for s in cn_sents])

cn2idx = {n: i for i, n in enumerate(cn_chars)}
en2idx = {n: i for i, n in enumerate(en_chars)}

print('max_encoder_seq_len :',max_encoder_seq_len)
print('max_decoder_seq_len :',max_decoder_seq_len)

max_encoder_seq_len : 38
max_decoder_seq_len : 49


In [ ]:
n_step = 55
batch_size = 64
n_hidden = 128

def make_data(seq_data):
    enc_input_all, dec_input_all, dec_output_all = [], [], []

    for seq in seq_data:
        # for i in range(2):
            # seq[i] = seq[i] + ' <PAD>' * (n_step - len(seq[i])) 
        enc_input = [en2idx[n] for n in (seq[0] + ' <EOS>').split()]
        
        dec_input = []
        dec_input.append(cn2idx['<SOS>'])
        dec_input.extend([cn2idx[n] for n in seq[1]])

        dec_output = [cn2idx[n] for n in seq[1]] 
        dec_output.append(cn2idx['<EOS>'])

        for i in range(n_step - len(enc_input)):
            enc_input.append(en2idx['<PAD>'])
        for i in range(n_step - len(dec_input)):
            dec_input.append(cn2idx['<PAD>'])
        for i in range(n_step - len(dec_output)):
            dec_output.append(cn2idx['<PAD>'])

        enc_input_all.append(np.eye(num_encoder_tokens)[enc_input])
        dec_input_all.append(np.eye(num_decoder_tokens)[dec_input])
        dec_output_all.append(dec_output) 


    # make tensor
    return torch.Tensor(enc_input_all), torch.Tensor(dec_input_all), torch.LongTensor(dec_output_all)

enc_input_all, dec_input_all, dec_output_all = make_data(pairs)

In [ ]:
class TranslateDataSet(Data.Dataset):
    def __init__(self, enc_input_all, dec_input_all, dec_output_all):
        self.enc_input_all = enc_input_all
        self.dec_input_all = dec_input_all
        self.dec_output_all = dec_output_all
    
    def __len__(self): # return dataset size
        return len(self.enc_input_all)
    
    def __getitem__(self, idx):
        return self.enc_input_all[idx], self.dec_input_all[idx], self.dec_output_all[idx]

loader = Data.DataLoader(TranslateDataSet(enc_input_all, dec_input_all, dec_output_all), batch_size, True)

In [ ]:
# Model
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.GRU(input_size=num_encoder_tokens, hidden_size=n_hidden,dropout=0.5) # encoder
        self.decoder = nn.GRU(input_size=num_decoder_tokens, hidden_size=n_hidden,dropout=0.5) # decoder
        self.fc = nn.Linear(n_hidden, num_decoder_tokens)

    def forward(self, enc_input, enc_hidden, dec_input):
        # enc_input(=input_batch): [batch_size, n_step+1, n_class]
        # dec_inpu(=output_batch): [batch_size, n_step+1, n_class]
        enc_input = enc_input.transpose(0, 1) # enc_input: [n_step+1, batch_size, n_class]
        dec_input = dec_input.transpose(0, 1) # dec_input: [n_step+1, batch_size, n_class]
        # h_t : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        _, h_t= self.encoder(enc_input, enc_hidden)
        # outputs : [n_step+1, batch_size, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.decoder(dec_input,h_t)

        model = self.fc(outputs) # model : [n_step+1, batch_size, n_class]
        return model

In [ ]:
model = Seq2Seq().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5000):
  for enc_input_batch, dec_input_batch, dec_output_batch in loader:
      # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
      h_0 = torch.zeros(1, batch_size, n_hidden).to(device)

      (enc_input_batch, dec_intput_batch, dec_output_batch) = (enc_input_batch.to(device), dec_input_batch.to(device), dec_output_batch.to(device))
      # enc_input_batch : [batch_size, n_step+1, n_class]
      # dec_intput_batch : [batch_size, n_step+1, n_class]
      # dec_output_batch : [batch_size, n_step+1], not one-hot
      pred = model(enc_input_batch, h_0, dec_intput_batch)
      # pred : [n_step+1, batch_size, n_class]
      pred = pred.transpose(0, 1) # [batch_size, n_step+1(=6), n_class]
      loss = 0
      for i in range(len(dec_output_batch)):
          # pred[i] : [n_step+1, n_class]
          # dec_output_batch[i] : [n_step+1]
          loss += criterion(pred[i], dec_output_batch[i])
      if (epoch + 1) % 500 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
          
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 0500 cost = 16.709499
Epoch: 0500 cost = 18.907497
Epoch: 1000 cost = 1.098630
Epoch: 1000 cost = 1.082275
Epoch: 1500 cost = 0.235056
Epoch: 1500 cost = 0.208419
Epoch: 2000 cost = 0.090194
Epoch: 2000 cost = 0.093186
Epoch: 2500 cost = 0.045772
Epoch: 2500 cost = 0.043096
Epoch: 3000 cost = 0.023712
Epoch: 3000 cost = 0.022712
Epoch: 3500 cost = 0.012100
Epoch: 3500 cost = 0.013190
Epoch: 4000 cost = 0.007097
Epoch: 4000 cost = 0.007034
Epoch: 4500 cost = 0.003724
Epoch: 4500 cost = 0.004295
Epoch: 5000 cost = 0.002152
Epoch: 5000 cost = 0.002451


In [ ]:
idx2cn = {i: n for i, n in enumerate(cn_chars)}
# Test
def translate(word):
    enc_input, dec_input, _ = make_data([[word, '']])
    enc_input, dec_input = enc_input.to(device), dec_input.to(device)
    # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
    hidden = torch.zeros(1, 1, n_hidden).to(device)
    output = model(enc_input, hidden, dec_input)
    # output : [n_step+1, batch_size, n_class]

    predict = output.data.max(2, keepdim=True)[1] # select n_class dimension


    decoded = [idx2cn[i.item()] for i in predict]

    translated = ''.join(decoded[:decoded.index('<PAD>')])

    return translated

In [ ]:


# pairs[:5]
for sent in pairs:
    print("{} => {}".format(sent[1], translate(sent[0])))


大卫.盖罗：这位是比尔.兰格， 我是大卫.盖罗。 => 大卫一一的都在这那。
我们将用一些影片来讲述一些深海里的故事。 => 我们将用一中中一。。
我们这有不少精彩的泰坦尼克的影片， 可惜您今天看不到。 => 我们这这有有有团儿里
泰坦尼克号 是拿了不少票房冠军 但事实上它并不是关于海洋的最刺激的故事。 => 泰泰尼尼尼  的的是是－－－
原因在于我们一直没把海洋当回事儿。 => 原真以喜一，的的动这
大家想想，海洋占了地球面积的75％。 => 大家所，，的的都在虾太介。
地球的大部分都是海水。 => 地的的的动动
海洋的平均深度是两英里 => 海洋的的动动
其实地球上最长的山脉都在海洋里。 => 其其这的的的这。。
大部分的动物也都生活在海洋里。 => 大大一的的动
大多数地震和火山喷发也都发生在海洋里 在海洋的最底部。 => 大多这这的的在这。。
海洋里生物的多样性和密度要比 雨林带还高。 => 海洋这脉的的动动儿
这儿基本上都没有被开发过，但是像这些美丽的景色， 它吸引着我们并被我们所熟知。 => 这儿会的的的都都科。
但我想告诉你的是，当你站在海边时， 你面对的是一个完全陌生的世界。 => 但我一诉诉，，是是是－－－－－
我们得用非常特殊的仪器 才能到达那个陌生的世界。 => 我们得些些的的的会会
我们用的是深海潜水艇Alvin号和摄像机， 摄像机是比尔.兰格和索尼共同研发的。 => 我们用的的的动动动
马塞尔.普鲁斯特说过："真正的探索之旅 不是为了新的发现，而是为了找到新的视角。" => 马马塞想想所，的的－－－－－－
这是一只水母。 => 这是一的的
是我最喜欢的，因为它哪都能动。 => 是我最声一的的都－－样
原来它是海洋中最长的生物。 => 原它它是海洋长。
它可以伸展到150英尺长。 => 它可的的的动尺
看到这些在动的东西了吗？ => 看看这不的在在动动
我真喜欢这些东西。 => 我真真又一，度一科
底下这些都是鱼饵。它们上上下下的浮动。 => 底里这些都都在这深。。
还有这些摇晃着，旋转着的触角。 => 还有这这些虾动动
这是一种群栖动物。 => 这是是是动动部
其实它们都是由单独的动物， 结合在一起就成了这样大的一个生物。 => 其们会的的动动。。
还有前面的这个是推进引擎。 它一会儿会用到它，还有一些光。 => 还还这些还动动的。
如果你